In [10]:
SELECT * FROM 'AppleStore.csv'
LIMIT 10;

,column00,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices_num,ipadSc_urls_num,lang_num,vpp_lic
0,1,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,1
1,2,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,1
2,3,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,37,5,3,1
3,4,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,37,5,9,1
4,5,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,37,5,45,1
5,6,283619399,Shanghai Mahjong,10485713,USD,0.99,8253,5516,4.0,4.0,1.8,4+,Games,47,5,1,1
6,7,283646709,PayPal - Send and request money safely,227795968,USD,0.00,119487,879,4.0,4.5,6.12.0,4+,Finance,37,0,19,1
7,8,284035177,Pandora - Music & Radio,130242560,USD,0.00,1126879,3594,4.0,4.5,8.4.1,12+,Music,37,4,1,1
8,9,284666222,PCalc - The Best Calculator,49250304,USD,9.99,1117,4,4.5,5.0,3.6.6,4+,Utilities,37,5,1,1
9,10,284736660,Ms. PAC-MAN,70023168,USD,3.99,7885,40,4.0,4.0,4.0.4,4+,Games,38,0,10,1


In [11]:
SELECT * FROM 'appleStore_description.csv'
LIMIT 10;

,id,track_name,size_bytes,app_desc
0,281656475,PAC-MAN Premium,100788224,"SAVE 20%, now only $3.99 for a limited time!\n..."
1,281796108,Evernote - stay organized,158578688,Let Evernote change the way you organize your ...
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,Download the most popular free weather app pow...
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,The eBay app is the best way to find anything ...
4,282935706,Bible,92774400,On more than 250 million devices around the wo...
5,283619399,Shanghai Mahjong,10485713,"★ WINNER ""BEST GAME"" 2009\n★ 3rd PLACE WINNER ..."
6,283646709,PayPal - Send and request money safely,227795968,Description\nTAP INTO YOUR MONEY\nSend money o...
7,284035177,Pandora - Music & Radio,130242560,Find the music you love and let the music you ...
8,284666222,PCalc - The Best Calculator,49250304,"PCalc is the powerful choice for scientists, e..."
9,284736660,Ms. PAC-MAN,70023168,Now with MFi controller support!\n\nMs. PAC-MA...


In [4]:
-- Number of unique apps in both tables
SELECT COUNT(DISTINCT id) AS unique_app_ids
FROM appleStore_description.csv;

,UniqueAppIDs
0,7197


In [1]:
-- Check for missing values in key fields
SELECT COUNT(*) as missing_values
FROM AppleStore.csv
WHERE track_name is null or user_rating is null or prime_genre is NULL;

,MissingValues
0,0


In [2]:
-- Number of apps per genre
SELECT prime_genre, COUNT(*) as num_apps
FROM AppleStore.csv
GROUP BY prime_genre
ORDER BY num_apps DESC;

,prime_genre,NumApps
0,Games,3862
1,Entertainment,535
2,Education,453
3,Photo & Video,349
4,Utilities,248
5,Health & Fitness,180
6,Productivity,178
7,Social Networking,167
8,Lifestyle,144
9,Music,138


In [3]:
-- Overview of apps' ratings
SELECT min(user_rating) AS min_rating
	   , max(user_rating) as max_rating
       , avg(user_rating) as avg_rating
FROM AppleStore.csv;

,MinRating,MaxRating,AvgRating
0,0.0,5.0,3.526956


In [5]:
-- Data Analysis

-- Do paid apps have higher ratings than free apps?
SELECT CASE
			WHEN price > 0 then 'Paid'
            ELSE 'Free'
         END AS app_type
         ,avg(user_rating) as avg_rating
FROM AppleStore.csv
GROUP BY app_type;

,app_type,avg_rating
0,Paid,3.720949
1,Free,3.376726


In [6]:
-- Do apps with more supported languages have higher ratings?
SELECT CASE
			WHEN lang_num < 10 then '<10 languages'
            WHEN lang_num BETWEEN 10 and 30 then '10-30 languages'
            ELSE '>30 languages'
         END AS language_bucket
         ,avg(user_rating) as avg_rating
FROM AppleStore.csv         
GROUP BY language_bucket
ORDER BY avg_rating DESC;

,language_bucket,avg_rating
0,10-30 languages,4.130512
1,>30 languages,3.777778
2,<10 languages,3.368327


In [2]:
-- Genres with low ratings
SELECT prime_genre
	   , avg(user_rating) as avg_rating
FROM AppleStore.csv
GROUP BY prime_genre
ORDER BY avg_rating ASC;

,prime_genre,avg_rating
0,Catalogs,2.100000
1,Finance,2.432692
2,Book,2.477679
3,Navigation,2.684783
4,Lifestyle,2.805556
5,News,2.980000
6,Sports,2.982456
7,Social Networking,2.985030
8,Food & Drink,3.182540
9,Entertainment,3.246729


In [9]:
-- Is there a correlation between the length of the app description and the user rating?
SELECT CASE
			WHEN length(b.app_desc) < 500 then 'Short'
            WHEN length(b.app_desc) BETWEEN 500 AND 1000 THEN 'Medium'
			ELSE 'Long'
         END AS description_length_bucket
         ,avg(a.user_rating) as average_rating
FROM AppleStore.csv as a
JOIN appleStore_description.csv as b
ON a.id = b.id
GROUP BY description_length_bucket
ORDER BY average_rating DESC;

,description_length_bucket,average_rating
0,Long,3.855947
1,Medium,3.232809
2,Short,2.533613


In [3]:
-- Top-rated apps for each genre
SELECT prime_genre
    ,track_name
    ,user_rating
FROM (
  	 SELECT
		prime_genre
  	    ,track_name
  	    ,user_rating
  	    ,RANK() OVER(PARTITION BY prime_genre ORDER BY user_rating DESC, rating_count_tot DESC) as rank
  	 FROM AppleStore.csv
  ) as a
WHERE a.rank = 1;

,prime_genre,track_name,user_rating
0,Food & Drink,Domino's Pizza USA,5.0
1,News,The Guardian,5.0
2,Utilities,Flashlight Ⓞ,5.0
3,Entertainment,Bruh-Button,5.0
4,Catalogs,CPlus for Craigslist app - mobile classifieds,5.0
5,Reference,Sky Guide: View Stars Night or Day,5.0
6,Weather,"NOAA Hi-Def Radar Pro - Storm Warnings, Hurri...",5.0
7,Business,TurboScan™ Pro - document & receipt scanner: s...,5.0
8,Health & Fitness,Yoga Studio,5.0
9,Medical,Blink Health,5.0
